In [1]:
import numpy as np
import re
from functools import reduce
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import helpers

In [2]:
preproc.do_preprocessing('../twitter-datasets/train_pos.txt')
preproc.do_preprocessing('../twitter-datasets/train_neg.txt')

100%|██████████| 100000/100000 [00:40<00:00, 2447.50it/s]


In [3]:
start_time = time.time()

In [4]:
# False to return the processed small dataset, True for the big one
lines, y = preproc.return_processed_trainset_and_y(False)

len(lines)

200000

In [5]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,2), sublinear_tf=True, max_features=None)
X = vectorizer.fit_transform(lines)

In [6]:
clf = LogisticRegression().fit(X, y)
kf = KFold(n_splits=4, shuffle=True, random_state=0)
scores = cross_val_score(clf, X, y, cv=kf, n_jobs=-1)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.8245 (+/- 0.0029)


In [7]:
elapsed_time = divmod(round((time.time() - start_time)), 60)
print('------\nElapsed time: {m} min {s} sec\n'.format(m=elapsed_time[0], s=elapsed_time[1]))

------
Elapsed time: 0 min 29 sec

